# Introduction

Introduction where you discuss the business problem and who would be interested in this project
Recommend a new area to open IB school in Dubai based on schools proximity, School Fees, Dubai Rental values. Customer wants to know
5 possible locations to start a new IB school in Dubai. Best possible locations to be determined based on area cluster, curriculum
of the schools, afforability and access to certain important points like transportation, library etc.,

Data Collection:
    1. Bayut.com - Property search enterprise publishes report on rental values quarterly basis
    2. www.edarabia.com - Number 1 education guide lists down all schools and universities.
    3. KHDA.com - Knowledge and Human Development Authority sets Education fees by maintaining the education cost index. KHDA is
    responsible for regulating the quality of Dubai private education
    4. Foursquare - builds location-aware experiences with Foursquare technology and data.
    
Data Usage to solve the problem:
    1. Bayut - Lists down properties with rental values by area
    2. edarabia.com - Lists down all the schools by Name, curriculum
    3. KHDA.com - Lists down school details, ranking and fees details
    4. Geocoder - Geo Location is determined using geocoder python api to determine latitude and longitude
    5. Foursquare - Latitude and Longitude determined using geocoder is used to determine areas of interest around the school location.
    
Problem solving:
    1. Data Preparation and cleansing (will be explained in next assignment)
    2. Determination of Pearson correlation (will be explained in next assignment)
    3. Machine Learning to build recommendation system (will be explained in next assignment)

In [3]:
## Tried to extract using beautiful soup but crawling has been disabled.
##specify the url
#page = requests.get('https://www.edarabia.com/dubai-school-fees/')

## parse the html using beautiful soup and store in variable `soup`
#soup = BeautifulSoup(page.text, 'html.parser')
#school_fees_table =  soup.find(class_='table-responsive')

In [2]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='1009ba7b-5586-448b-b530-29a9ee8e12b7', project_access_token='p-b7fbc67668498bf0ac7f355da4dd7780cb2a6185')
pc = project.project_context

In [3]:
import pandas as pd
import numpy as np

In [6]:
my_file = project.get_file('Dubai_School_details_csv_saran.csv')
df_data_school_details = pd.read_csv(my_file,encoding='cp1252')
df_data_school_details.replace(np.nan, 0, inplace=True)
df_data_school_details.head()

,Name,CURR.,FS1,KG1/FS2,KG2/Y1,G1/Y2,G2/Y3,G3/Y4,G4/Y5,G5/Y6,G6/Y7,G7/Y8,G8/Y9,G9/Y10,G10/Y11,G11/Y12,G12/Y13
0,Adab Iranian Private School,Iranian,0.0,8778.0,8778.0,11284.0,11284.0,11284.0,11284.0,11284.0,13792.0,13792.0,13792.0,16300.0,16300.0,18808.0,18808.0
1,Al Ameen Private School,"British, IGCSE",9411.0,9411.0,10037.0,10037.0,10037.0,10037.0,10037.0,10037.0,10667.0,10667.0,10667.0,10667.0,10667.0,0.0,0.0
2,Al Arqam Private School,MOE,0.0,7791.0,8196.0,9095.0,9095.0,9095.0,9916.0,9916.0,9916.0,11597.0,11597.0,11597.0,14961.0,15788.0,16618.0
3,Al Diyafah High School,British,0.0,10701.0,11304.0,11304.0,11304.0,11634.0,11634.0,11634.0,11887.0,13523.0,14927.0,18922.0,18922.0,19232.0,22725.0
4,Al Eman School Dubai,MOE,0.0,5707.0,5707.0,5881.0,5881.0,5881.0,6918.0,6918.0,6918.0,8503.0,8503.0,8503.0,0.0,0.0,0.0


In [7]:
!pip install opencage

Requirement not upgraded as not directly required: six>=1.4.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from opencage)
Requirement not upgraded as not directly required: Requests>=2.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from opencage)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from Requests>=2.2.0->opencage)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from Requests>=2.2.0->opencage)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from Requests>=2.2.0->opencage)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from Requests>=2.2.0->opencage)


Data Preparation: 
Find Geo Code for the school names - OpenCageGeocode provides API to convert coordinates to and from places
Easy, Open, Worldwide, Affordable
Following code was executed to collect the latitude, longitude and area

In [8]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
import time

In [9]:
# @hidden_cell
key = '2700a7689bf64070b3e5d0ac19803c0d'

In [10]:
geocoder = OpenCageGeocode(key)
school_name = "Victory Heights Primary School, Dubai"
geocoder = OpenCageGeocode(key)
results = geocoder.geocode(school_name)
latitude = results[0]['geometry']['lat']
longitutde = results[0]['geometry']['lng']
print (latitude)
print (longitutde)
print(u'%f,%f,%s,%s,%s' % (results[0]['geometry']['lat'], 
                        results[0]['geometry']['lng'],
                        results[0]['components']['suburb'],
                        results[0]['components']['country_code'],
                        results[0]['annotations']['timezone']['name']))
#pprint(results)

25.0351877
55.2224797
25.035188,55.222480,Dubai Sports City,ae,Asia/Dubai


Data Preparation: Call the geocoder for all the schools. 
Some of the schools area was missing in the api and hence the data was prepared for the missing geo location

In [11]:
#def getLatLng(school_name):
#    results = geocoder.geocode(school_name)
#    latitude = results[0]['geometry']['lat']
#    longitutde = results[0]['geometry']['lng']
#    area = results[0]['components']['suburb']
#    return (latitude, longitutde)

In [12]:
#for index, row in df_data_school_details.iterrows():
#    print(row['Name'])
#    (lat,lng) = getLatLng(row['Name'])
#    df_data_school_details['latitude'] = lat
#    df_data_school_details['longitutde'] = lng
#    df_data_school_details['area'] = area

In [13]:
my_file = project.get_file('Dubai_School_details_saran_v2.csv')
df_data_school_details_geo = pd.read_csv(my_file,encoding='cp1252')
df_data_school_details_geo.replace(np.nan, 0, inplace=True)
df_data_school_details_geo.head()

,Name,Latitude,longitutude,area,Curriculum-1,Curriculum-2,FS1,KG1-FS2,KG2-Y1,G1-Y2,...,G6-Y7,G7-Y8,G8-Y9,G9-Y10,G10-Y11,G11-Y12,G12-Y13,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Al Arqam Private School,25.108474,55.189122,Al Barsha,MOE,0,0.0,7791.0,8196.0,9095.0,...,9916.0,11597.0,11597.0,11597.0,14961.0,15788.0,16618.0,0.0,0.0,0.0
1,Al Diyafah High School,25.290112,55.374734,Al Nahda 2,British,0,0.0,10701.0,11304.0,11304.0,...,11887.0,13523.0,14927.0,18922.0,18922.0,19232.0,22725.0,0.0,0.0,0.0
2,Al Eman School Dubai,25.230753,55.389201,Al Khawaneej 2,MOE,0,5707.0,5707.0,5881.0,5881.0,...,8503.0,8503.0,8503.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Al Ittihad Private School,Al Mamzar",25.300585,55.344829,Al Mamzar,American,0,18180.0,17495.0,17495.0,20290.0,...,22115.0,28275.0,28275.0,28275.0,35675.0,35675.0,40865.0,0.0,0.0,0.0
4,"Al Ittihad Private School,Jumeirah",25.180685,55.240945,Al Quoz,American,0,0.0,21145.0,21145.0,28195.0,...,33060.0,35005.0,35005.0,35005.0,43145.0,43145.0,43145.0,0.0,0.0,0.0


Data Preparation: Area Names are normalized to match the rental data available.

In [14]:
my_file = project.get_file('Dubai_School_details_saran_v3.csv')
df_data_school_details_geo_area = pd.read_csv(my_file,encoding='cp1252')
df_data_school_details_geo_area.replace(np.nan, 0, inplace=True)
df_data_school_details_geo_area.head()

,Name,Latitude,longitutude,area,Curriculum-1,Curriculum-2,FS1,KG1-FS2,KG2-Y1,G1-Y2,...,G6-Y7,G7-Y8,G8-Y9,G9-Y10,G10-Y11,G11-Y12,G12-Y13,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Al Arqam Private School,25.108474,55.189122,Al Barsha,MOE,0,0.0,7791.0,8196.0,9095.0,...,9916.0,11597.0,11597.0,11597.0,14961.0,15788.0,16618.0,0.0,0.0,0.0
1,Al Diyafah High School,25.290112,55.374734,Al Nahda,British,0,0.0,10701.0,11304.0,11304.0,...,11887.0,13523.0,14927.0,18922.0,18922.0,19232.0,22725.0,0.0,0.0,0.0
2,Al Eman School Dubai,25.230753,55.389201,Al Khawaneej,MOE,0,5707.0,5707.0,5881.0,5881.0,...,8503.0,8503.0,8503.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Al Ittihad Private School,Al Mamzar",25.300585,55.344829,Al Mamzar,American,0,18180.0,17495.0,17495.0,20290.0,...,22115.0,28275.0,28275.0,28275.0,35675.0,35675.0,40865.0,0.0,0.0,0.0
4,"Al Ittihad Private School,Jumeirah",25.180685,55.240945,Al Quoz,American,0,0.0,21145.0,21145.0,28195.0,...,33060.0,35005.0,35005.0,35005.0,43145.0,43145.0,43145.0,0.0,0.0,0.0


In [15]:
df_data_school_details_geo_area.shape

(191, 24)

In [42]:
my_file = project.get_file('Dubai_rent_2018_saran.csv')
df_data_dubai_rent = pd.read_csv(my_file,encoding='cp1252')
df_data_dubai_rent.replace(np.nan, 0, inplace=True)
df_data_dubai_rent.head()

,Area,Apartments - 1 BR,Apartments - 2 BR,Apartments - 3 BR,Villas - 3 BR,Villas - 4 BR
0,Al Barsha,"68,968","91,587","112,190","165,000","200,000"
1,Al Karama,"72,166","95,101","109,728",0,"150,000"
2,Al Quoz,"54,021","72,226","127,500","142,500","179,600"
3,Al Qusais,"51,137","70,385","86,800","82,000",0
4,Al Sufouh,"81,929","107,000","165,000","245,417","260,000"


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.19.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [2]:
# @hidden_cell
CLIENT_ID = '2NJKFZUIRT2ENHQC2N0W33C5JIYUL1KQ15NCONHIN1O2PTML' # your Foursquare ID
CLIENT_SECRET = 'A5ZCGMFV5QQ5QJAWNSPAQTFF0FV4G2XYVSWN2FHINC4M2EZD' # your Foursquare Secret
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [3]:
VERSION = '20181206'

In [4]:
#address = 'World Trade Centre Metro Station, Dubai'
#geolocator = Nominatim(user_agent="foursquare_agent")
#location = geolocator.geocode(address)
#latitude = location.latitude
latitude = 25.099977
#longitude = location.longitude
longitude = 55.182493
print(latitude, longitude)

25.099977 55.182493


In [5]:
search_query = 'Hospital'
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&search_query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query,radius, LIMIT)
#url

In [7]:
results = requests.get(url).json()

results

{'meta': {'code': 200, 'requestId': '5cd87ed9351e3d1d4f7b16b6'},
 'response': {'confident': False,
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d13d941735',
      'name': 'High School',
      'pluralName': 'High Schools',
      'primary': True,
      'shortName': 'High School'}],
    'hasPerk': False,
    'id': '4cda1012d6656a31cd58fb3e',
    'location': {'address': 'Intersection of Road 316 and 331',
     'cc': 'AE',
     'city': 'دبي',
     'country': 'الإمارات العربية المتحدة',
     'distance': 196,
     'formattedAddress': ['Intersection of Road 316 and 331',
      'دبي',
      'الإمارات العربية المتحدة'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.09823014077149,
       'lng': 55.182223320007324}],
     'lat': 25.09823014077149,
     'lng': 55.182223320007324,
     'state': 'دبي'},
    'name': 'The American School of Dubai',
    'referralId': 'v-15

In [8]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.count()

categories                   50
hasPerk                      50
id                           50
location.address             24
location.cc                  50
location.city                35
location.country             50
location.crossStreet         11
location.distance            50
location.formattedAddress    50
location.labeledLatLngs      50
location.lat                 50
location.lng                 50
location.neighborhood        14
location.postalCode           3
location.state               35
name                         50
referralId                   50
dtype: int64

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The American School of Dubai,High School,Intersection of Road 316 and 331,AE,دبي,الإمارات العربية المتحدة,NaN,196,"[Intersection of Road 316 and 331, دبي, الإمار...","[{'lng': 55.182223320007324, 'label': 'display...",25.098230,55.182223,NaN,NaN,دبي,4cda1012d6656a31cd58fb3e
1,High School Gym @ ASD,Athletics & Sports,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,106,[الإمارات العربية المتحدة],"[{'lng': 55.18282708665772, 'label': 'display'...",25.100887,55.182827,NaN,NaN,NaN,50b9dc1be4b0e9eceaf7b60d
2,C-207 Best Classroom On Earth,Middle School,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,39,[الإمارات العربية المتحدة],"[{'lng': 55.18237957241771, 'label': 'display'...",25.099637,55.182380,NaN,NaN,NaN,5227f2f711d23e48434c8dad
3,Kung Korean Restaurant and Karaoke,Karaoke Bar,Mezzanine Floor of Byblos Hotel ; Tecom Al Barsha,AE,دبي,الإمارات العربية المتحدة,NaN,433,[Mezzanine Floor of Byblos Hotel ; Tecom Al Ba...,"[{'lng': 55.178194642066956, 'label': 'display...",25.099894,55.178195,Barsha Heights,NaN,دبي,4e53792181302f16caf97ec6
4,API Building,Building,"Al Barsha, near Ibis hotel",AE,NaN,الإمارات العربية المتحدة,Service Road,42,"[Al Barsha, near Ibis hotel (Service Road), ال...","[{'lng': 55.182104594949685, 'label': 'display...",25.100132,55.182105,NaN,NaN,NaN,5146fd19e4b0702d34e4b69d
5,Dubai Tennis Stadium,Tennis Court,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,212,"[دبي, الإمارات العربية المتحدة]","[{'lng': 55.182503582153316, 'label': 'display...",25.098067,55.182504,NaN,NaN,دبي,5124c882e4b08cbcc1e721ee
6,Leader building,None,Tecom,AE,دبي,الإمارات العربية المتحدة,NaN,440,"[Tecom, دبي, الإمارات العربية المتحدة]","[{'lng': 55.179004, 'label': 'display', 'lat':...",25.097594,55.179004,NaN,NaN,دبي,4c1065e23ce120a19b1e081c
7,Bollywood Cafe,Speakeasy,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,273,"[دبي, الإمارات العربية المتحدة]","[{'lng': 55.1797915841854, 'label': 'display',...",25.099769,55.179792,NaN,NaN,دبي,4dbb355c5da3ff58ec2bc439
8,Ramee Rose Hotel,Hotel,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,368,"[دبي, الإمارات العربية المتحدة]","[{'lng': 55.179462699623016, 'label': 'display...",25.098124,55.179463,Barsha Heights,NaN,دبي,5158aea4e4b02bc223979c75
9,Asd Falcon Aquatic Nest,Middle School,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,114,[الإمارات العربية المتحدة],"[{'lng': 55.18210459494969, 'label': 'display'...",25.099010,55.182105,NaN,NaN,NaN,523d2c8e11d2beb09d952a5c


In [40]:
#dataframe_filtered = dataframe_filtered.head(30)
dataframe_filtered.head(30)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,state,id
0,Blue Waves Residence 2,Building,NaN,AE,البرشاء,الإمارات العربية المتحدة,NaN,405,"[البرشاء, الإمارات العربية المتحدة]","[{'label': 'display', 'lng': 55.186684, 'lat':...",25.111369,55.186684,NaN,دبي,5c2b8509d905160026a5d48a
1,Ban Khun Mae - Authentic Thai Cuisine,Thai Restaurant,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,485,[الإمارات العربية المتحدة],"[{'label': 'display', 'lng': 55.18604602312715...",25.111827,55.186046,NaN,NaN,4ef2f0a030f8e7873c6d4511
2,Al Barsha 1 Park 1,Park,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,217,"[دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lng': 55.1871352049838,...",25.107706,55.187135,NaN,دبي,598b6573f0ca9506703f08de
3,Al Arqam School,General College & University,Barsha,AE,دبي,الإمارات العربية المتحدة,NaN,77,"[Barsha, دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lng': 55.18881753279272...",25.109112,55.188818,NaN,دبي,4cad803518a3199c13845bfb
4,M&N Cafe & Brasserie,Café,NaN,AE,Istanbul,الإمارات العربية المتحدة,NaN,462,"[Istanbul, الإمارات العربية المتحدة]","[{'label': 'display', 'lng': 55.18778764226069...",25.112445,55.187788,NaN,Istanbul,4cb08afb39458cfad99903a0
5,Al Barsha Mosque,Mosque,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,321,"[دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lng': 55.187718, 'lat':...",25.111069,55.187718,NaN,دبي,58d4e634fa7ed028d22ac651
6,Dubai National School,High School,Al Barsha 1,AE,دبي,الإمارات العربية المتحدة,NaN,483,"[Al Barsha 1, دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lng': 55.19361127014419...",25.106951,55.193611,Al Barsha 1,دبي,4d2a81bc8292236a516b2cbb
7,Nom Nom Asia Restaurant,Asian Restaurant,"shop 1, Barsha Oasis Building, Behind Novotel ...",AE,دبي,الإمارات العربية المتحدة,10 B,217,"[shop 1, Barsha Oasis Building, Behind Novotel...","[{'label': 'display', 'lng': 55.19121167176751...",25.107986,55.191212,Al Barsha 1,دبي,5438262e498e8986490ffb09
8,ICE-WATCH KIOSK MOE,Jewelry Store,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,236,"[دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lng': 55.1871405969578,...",25.109619,55.187141,NaN,دبي,4ccecc55c0378cfacedb9548
9,Parking Level 3,Parking,Dubai Mall,AE,دبي,الإمارات العربية المتحدة,NaN,218,"[Dubai Mall, دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lng': 55.19100676536067...",25.109447,55.191007,Al Barsha 1,دبي,4bf7bb18508c0f4798ea3d31


In [1]:
# Install FourSquare client library
!pip install foursquare

Requirement not upgraded as not directly required: requests>=2.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from foursquare)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from foursquare)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests>=2.1->foursquare)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests>=2.1->foursquare)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests>=2.1->foursquare)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests>=2.1->foursquare)
  Running setup.py bdist_wheel for foursquare ... done
  Stored in dir

In [2]:
# @hidden_cell
CLIENT_ID = '2NJKFZUIRT2ENHQC2N0W33C5JIYUL1KQ15NCONHIN1O2PTML' # your Foursquare ID
CLIENT_SECRET = 'A5ZCGMFV5QQ5QJAWNSPAQTFF0FV4G2XYVSWN2FHINC4M2EZD' # your Foursquare Secret

In [3]:
import foursquare
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
fs = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

In [4]:
RADIUS = 1500 # 500m, around 5 minutes walking time

In [11]:
# Define a function to search nearby information and convert the result as dataframe
def venues_nearby(latitude, longitude, category, verbose=True):    
    results = fs.venues.search(
        params = {
            'query': category, 
            'll': '{},{}'.format(latitude, longitude),
            'radius': RADIUS,
            'category': category
        }
    )    
    df = json_normalize(results['venues'])
    cols = ['Name','Latitude','Longitude']    
    if( len(df) == 0 ):        
        df = pd.DataFrame(columns=cols)
    else:        
        df = df[['name','location.lat','location.lng']]
        df.columns = cols
    if( verbose ):
        print('{} "{}" venues are found within {}m of location'.format(len(df), category, RADIUS))
    return df

In [12]:
latitude = 25.099977
#longitude = location.longitude
longitude = 55.182493

In [13]:
venues_nearby(latitude,longitude, 'Tennis Court').head()

12 "Tennis Court" venues are found within 1500m of location


,Name,Latitude,Longitude
0,American School of Dubai Tennis Courts,25.102948,55.178341
1,Squash Court @ Quay Club - Madinat Jumeirah,25.108517,55.183781
2,Dubai Tennis Stadium,25.098067,55.182504
3,Elementary Basketball Court @ ASD,25.101628,55.183258
4,Tennis @ The Greens,25.092385,55.167966


In [16]:
venues_nearby(latitude, longitude, 'Sports').head()

5 "Sports" venues are found within 1500m of location


,Name,Latitude,Longitude
0,Sports Stars Media,25.104109,55.171506
1,Goodfellas Sports Lounge,25.097922,55.179149
2,Aqua Sports Academy,25.095127,55.174406
3,Cycle Sports,25.112328,55.191829
4,Good Fellas,25.098266,55.179569


In [15]:
venues_nearby(latitude, longitude, 'Garden').head()

7 "Garden" venues are found within 1500m of location


,Name,Latitude,Longitude
0,Garden Restaurant at Atana Hotel,25.102201,55.178585
1,dubai garden glow,25.101242,55.179553
2,Dubai Garden Centre,25.112142,55.187071
3,China Garden Restaurant,25.112255,55.191864
4,Gardens pharmacy,25.093482,55.168418


In [41]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=50) 

# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)



# display map
venues_map

NameError: name 'folium' is not defined

## Methodology

With above data, I can build a content-based recommender systems to provide recommendation by comibining Average rental, average school fees, cluster.

Combine with FourSquare API on counting how many different venues (Food, pharmacy, metro station, library, martial arts) to determine stake holders favorite list.


In [4]:
my_file = project.get_file('Dubai_rent_2018_saran_v3.csv')
df_data_school_details_venues = pd.read_csv(my_file,encoding='cp1252')
df_data_school_details_venues.replace(np.nan, 0, inplace=True)
df_data_school_details_venues.head()

,Area,No of schools,Average Fees,No of IB Schools,No of Metro station,No of bus stop,Tennis Court,Garden,Athletics & Sports
0,Al Barsha,17,51477,3,2,5,3,2,1
1,Al Karama,12,16275,0,2,5,0,2,1
2,Al Quoz,15,33390,2,0,9,2,2,1
3,Al Qusais/ Muhaisnah,33,15533,0,2,10,2,2,1
4,Al Sufouh,3,59457,0,0,5,2,2,1


In [5]:
my_file = project.get_file('Dubai_hospitals_v1.0.csv_shaped.csv')
df_data_dubai_hospitals = pd.read_csv(my_file,encoding='cp1252')
df_data_dubai_hospitals.head()

,f_name_english,facilitysubcategorynameenglish,x_coordinate,y_coordinate,area_english,area_norm
0,DR ADEL CLINIC PHYSIOTHERAPY,Rehabilitation Center,25.18347,55.226229,NaN,NaN
1,Reborn Therapeutic Massage Center,Rehabilitation Center,0.00000,0.000000,NaN,NaN
2,MOONFLOWER THERAPEUTIC MASSAGE CENTER,Beauty Center Salon,0.00000,0.000000,NaN,NaN
3,Beauty By Design Ploy Clinic L.L.C,PolyClinic,0.00000,0.000000,NaN,NaN
4,Kids Spot Nursery (Branch),School clinic,0.00000,0.000000,NaN,NaN


In [6]:
df_data_dubai_hospitals_shaped=df_data_dubai_hospitals.dropna(axis=0)
df_data_dubai_hospitals_shaped.head()

,f_name_english,facilitysubcategorynameenglish,x_coordinate,y_coordinate,area_english,area_norm
9,Arab Euro Home Nursing Services,Home Healthcare Agency,25.222300,55.281300,Bur Dubai,Bur Dubai
10,BOOTS PHARMACY DUBAI MALL 1,Community (Out Patient ) Pharmacy,25.196100,55.287985,Bur Dubai,Bur Dubai
11,Department of Finance,School clinic,0.000000,0.000000,Bur Dubai,Bur Dubai
12,Mega Scan Centre,Radio-Diagnostic Center,25.170577,55.150404,Bur Dubai,Bur Dubai
13,Midas Touch Medspa LLC,Specialty Clinic,25.199057,55.265562,Bur Dubai,Bur Dubai


In [7]:
df_data_dubai_hospitals_count = df_data_dubai_hospitals_shaped['area_norm'].value_counts()\
                 .to_frame('count').rename_axis('area_norm')\
                 .reset_index()

In [8]:
df_data_dubai_hospitals_count.rename(columns={'area_norm':'Area'}, inplace=True)
df_data_dubai_hospitals_count

,Area,count
0,Jumeirah,177
1,Al Qusais/ Muhaisnah,82
2,Al Marqabat,73
3,Al Karama,66
4,Umm Suqeim,53
5,Al Quoz,44
6,Al Barsha,43
7,Mirdif/ Al Rashidiya,41
8,Al Bada,39
9,Discovery Gardens/ Jebel Ali,39


In [9]:
my_file = project.get_file('Dubai_School_details_saran_v4.csv')
df_data_school_details_venues = pd.read_csv(my_file,encoding='cp1252')
df_data_school_details_venues.replace(np.nan, 0, inplace=True)
df_data_school_details_venues.sort_values(by=['Area'])
df_data_school_details_venues["area_id"] = df_data_school_details_venues.index + 1
#df_data_school_details_venues['Average_Fees']=df_data_school_details_venues['Average_Fees']/max(df_data_school_details_venues['Average_Fees'])
df_data_school_details_venues.head()


,Area,Schools_Count,Average_Fees,IB_Schools_count,Metro_count,Bus_stop_count,Hospital_count,Tennis_Court_count,Garden_count,Sports_center_count,area_id
0,Al Barsha,17,51477,3,2,5,43,3,2,1,1
1,Al Karama,12,16275,0,2,5,66,0,2,1,2
2,Al Quoz,15,33390,2,0,9,44,2,2,1,3
3,Al Qusais/ Muhaisnah,33,15533,0,2,10,82,2,2,1,4
4,Al Sufouh,3,59457,0,0,5,14,2,2,1,5


In [10]:
df_data_school_details_venues.corr()

,Schools_Count,Average_Fees,IB_Schools_count,Metro_count,Bus_stop_count,Hospital_count,Tennis_Court_count,Garden_count,Sports_center_count,area_id
Schools_Count,1.000000,-0.612517,0.216009,0.410423,0.450020,0.302686,0.301511,-0.281636,-0.410074,-0.306304
Average_Fees,-0.612517,1.000000,0.141283,-0.648360,-0.644970,-0.169970,0.208919,-0.067928,0.113552,0.445679
IB_Schools_count,0.216009,0.141283,1.000000,0.014243,0.010611,0.081459,0.537406,-0.205883,-0.202653,-0.124550
Metro_count,0.410423,-0.648360,0.014243,1.000000,0.655417,0.090404,-0.079603,-0.145852,0.140407,-0.301893
Bus_stop_count,0.450020,-0.644970,0.010611,0.655417,1.000000,0.051137,0.096115,0.079026,-0.032463,-0.343616
Hospital_count,0.302686,-0.169970,0.081459,0.090404,0.051137,1.000000,0.139861,-0.142997,-0.331662,0.028995
Tennis_Court_count,0.301511,0.208919,0.537406,-0.079603,0.096115,0.139861,1.000000,-0.094884,-0.447258,-0.098198
Garden_count,-0.281636,-0.067928,-0.205883,-0.145852,0.079026,-0.142997,-0.094884,1.000000,0.167360,0.000000
Sports_center_count,-0.410074,0.113552,-0.202653,0.140407,-0.032463,-0.331662,-0.447258,0.167360,1.000000,0.384900
area_id,-0.306304,0.445679,-0.124550,-0.301893,-0.343616,0.028995,-0.098198,0.000000,0.384900,1.000000


In [11]:
my_file = project.get_file('Dubai_rent_2018_saran_v4.csv')
df_data_school_details_rent = pd.read_csv(my_file,encoding='cp1252')
df_data_school_details_rent.replace(np.nan, 0, inplace=True)
df_data_school_details_rent.sort_values(by=['Area'])
df_data_school_details_rent["area_id"] = df_data_school_details_rent.index + 1
df_data_school_details_rent.head()

,Area,Apartments - 1 BR,Apartments - 2 BR,Apartments - 3 BR,Villas - 3 BR,Villas - 4 BR,AVG_Rent,area_id
0,Al Barsha,"68,968","91,587","112,190","165,000","200,000",127549.00,1
1,Al Karama,"72,166","95,101","109,728",0,"150,000",106748.75,2
2,Al Quoz,"54,021","72,226","127,500","142,500","179,600",115169.40,3
3,Al Qusais/ Muhaisnah,"51,137","70,385","86,800","82,000",0,72580.50,4
4,Al Sufouh,"81,929","107,000","165,000","245,417","260,000",171869.20,5


In [12]:
df_data_school_details_rent_1 = df_data_school_details_rent.drop('Apartments - 1 BR', 1).drop('Apartments - 2 BR', 1).drop('Apartments - 3 BR', 1).drop('Villas - 3 BR', 1).drop('Villas - 4 BR', 1)
df_data_school_details_rent_1

,Area,AVG_Rent,area_id
0,Al Barsha,127549.00,1
1,Al Karama,106748.75,2
2,Al Quoz,115169.40,3
3,Al Qusais/ Muhaisnah,72580.50,4
4,Al Sufouh,171869.20,5
5,Arabian Ranches,151949.75,6
6,Bur Dubai,112766.75,7
7,Deira,113586.80,8
8,Discovery Gardens/ Jebel Ali,74295.00,9
9,Dubai Silicon Oasis/ Academic City,68915.00,10


In [13]:
#df_data_school_details_rent_1['AVG_Rent']=df_data_school_details_rent_1['AVG_Rent']/max(df_data_school_details_rent_1['AVG_Rent'])
df_data_school_details_rent_1

,Area,AVG_Rent,area_id
0,Al Barsha,127549.00,1
1,Al Karama,106748.75,2
2,Al Quoz,115169.40,3
3,Al Qusais/ Muhaisnah,72580.50,4
4,Al Sufouh,171869.20,5
5,Arabian Ranches,151949.75,6
6,Bur Dubai,112766.75,7
7,Deira,113586.80,8
8,Discovery Gardens/ Jebel Ali,74295.00,9
9,Dubai Silicon Oasis/ Academic City,68915.00,10


In [14]:
df_data_school_details_venues
df_data_school_details_rent_1


,Area,AVG_Rent,area_id
0,Al Barsha,127549.00,1
1,Al Karama,106748.75,2
2,Al Quoz,115169.40,3
3,Al Qusais/ Muhaisnah,72580.50,4
4,Al Sufouh,171869.20,5
5,Arabian Ranches,151949.75,6
6,Bur Dubai,112766.75,7
7,Deira,113586.80,8
8,Discovery Gardens/ Jebel Ali,74295.00,9
9,Dubai Silicon Oasis/ Academic City,68915.00,10


In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=50) 

# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)



# display map
venues_map

## Machine Learning
Now, let's use Content-Based systems. In this case, I am going to assign a weigthed average for the area based on the rents and calculating the total weights for the areas based on the venues.

Normalize the values of venues dataframe by using MinMaxScaler method

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [16]:
df_data_school_details_venues.drop(columns=['Area','area_id'], inplace=True)

In [17]:
venues_normalized = scaler.fit_transform(df_data_school_details_venues)

In [18]:
df_venues_normalized = pd.DataFrame(
    venues_normalized,
    index=df_data_school_details_venues.index,
    columns=df_data_school_details_venues.columns
)

In [19]:
df_venues_normalized.head()

,Schools_Count,Average_Fees,IB_Schools_count,Metro_count,Bus_stop_count,Hospital_count,Tennis_Court_count,Garden_count,Sports_center_count
0,0.50000,0.738982,1.000000,0.5,0.230769,0.100467,1.000000,0.0,0.0
1,0.34375,0.067073,0.000000,0.5,0.230769,0.154206,0.000000,0.0,0.0
2,0.43750,0.393751,0.666667,0.0,0.538462,0.102804,0.666667,0.0,0.0
3,1.00000,0.052910,0.000000,0.5,0.615385,0.191589,0.666667,0.0,0.0
4,0.06250,0.891298,0.000000,0.0,0.230769,0.032710,0.666667,0.0,0.0


Normalize the values of Area - Rental dataframe by using MinMaxScaler method

In [20]:
df_data_school_details_rent_1.drop(columns=['Area','area_id'], inplace=True)
rents_normalized = scaler.fit_transform(df_data_school_details_rent_1)

In [21]:
df_rents_normalized = pd.DataFrame(
    rents_normalized,
    index=df_data_school_details_rent_1.index,
    columns=df_data_school_details_rent_1.columns
)

In [22]:
#df_data_school_details_rent_1
rents_normalized

array([[ 0.37970962],
       [ 0.24500868],
       [ 0.29954021],
       [ 0.02373752],
       [ 0.66672409],
       [ 0.53772714],
       [ 0.28398082],
       [ 0.2892914 ],
       [ 0.0348405 ],
       [ 0.        ],
       [ 0.21019246],
       [ 0.29980184],
       [ 0.76772008],
       [ 0.25811628],
       [ 0.2105467 ],
       [ 0.50028624],
       [ 1.        ],
       [ 0.89746662],
       [ 0.81800211]])

## Results
Results section where you discuss the results.

With Area-Rental and Area-venues in hand, I am going to take the weighted average of every area and recommend the most probable area that most satisfy it.

In [27]:
# = (df_venues_normalized.sum(axis=1))*(df_data_school_details_rent_1.sum(axis=1))
df_recommend = (df_venues_normalized.sum(axis=1))*(rents_normalized.sum(axis=1))

In [28]:
df_recommend = df_recommend.reset_index(name='Rating')

In [29]:
df_recommend.sort_values('Rating', ascending=False, inplace=True)

In [30]:
df_recommend

,index,Rating
17,17,3.460101
12,12,2.714358
16,16,2.699976
18,18,2.559456
0,0,1.545501
5,5,1.381535
7,7,1.319385
4,4,1.256071
6,6,0.851974
2,2,0.840465


## Discussion

Although data collected is to best of available data from right sources, some of the data needs to be right validated. Foursquare API does not have data
for all schools. The weigthed average is calculated based on available data and the order of recommendation is provided.

## Conclusion

Conclusion section where you conclude the report.

The stakeholder problem of recommendation of areas where the "IB schools can be started" is resolved.

Advantages and Disadvantages of Content-Based Filtering
Advantages

    Learns existing setup
    Highly personalized current available schools

Disadvantages

    Doesnt take into account what others think of the as parameters to start the school, so low quality item recommendations might happen
    Extracting data is not always intuitive
    Determining what characteristics of the school the stakeholder dislikes or likes is not always obvious



